#### Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from IPython.display import clear_output


#### Load the source CSV file names into a list as a file references

In [2]:
path='../Resources/'
extract_table_names=pd.read_csv(path+'Extract_table_names.txt',header=None)



#### Read the uploaded file names list and store into Pandas data frame list

In [3]:
extract_df=[]
for index,table in extract_table_names.iterrows():
    extract_df.append(pd.read_csv(path+table[0]))



#### Function to print the content of data frame passing a dataframe index as (number)

In [4]:

 def print_extract_df(number):
     print(f"the table code is ({number}) the table name is: {extract_table_names[0][number]}")
     display(extract_df[number])
    
     return


#### Heatmap function to draw the data frame by a dataframe index

In [5]:
def check_extract_data(number):
    print(f"the table code is ({number}) the table name is: {extract_table_names[0][number]}")
    fig, ax = plt.subplots(figsize=(20,10))     
    cols = extract_df[number].columns 
    plt.title(extract_table_names[0][number])
    colours = ['#5b6180', '#FF6663'] 
    sns.heatmap(extract_df[number][cols].isnull(), cmap=sns.color_palette(colours))
    ax.set_xticklabels(ax.get_xticklabels(),rotation =90)
    plt.show()


#### Display the dataframe information summary by a dataframe index

In [6]:
def get_coloumns_info(number):
    print(f"the table code is ({number}) the table name is: {extract_table_names[0][number]}")
    extract_df[number].info()
    return 

#### Drop a list of columns from a dataframe by sending a list of cloumn index and dataframe index


In [7]:
def drop_columns(number,cols):
    print(f"the table code is ({number}) the table name is: {extract_table_names[0][number]}")
    extract_df[number].drop(extract_df[number].columns[cols],axis=1,inplace=True)
    return



#### Replace the nill value for a dataframe by sending the dataframe index


In [8]:
def replace_none(number):
    print(f"the table code is ({number}) the table name is: {extract_table_names[0][number]}")
    extract_df[number]=extract_df[number].fillna(0)
    return


#### Replacing the columns data type by sending a dictionary list for the column index and new data type by dataframe index


In [9]:
def replace_column_type(number,data):
    data_list=data.split(',')
    for cast in data_list:
        cs=cast.split(':')
        colname = extract_df[number].columns[int(cs[0])]
        extract_df[number][colname]=extract_df[number][colname].astype(cs[1], errors='raise')
    print(data_list)
    get_coloumns_info(number)
    return

#### Rename list of columns by sending a dictionary list of cloumn index and new name by dataframe index


In [10]:
def rename_column(number,data):
    data_list=data.split(',')
    for cast in data_list:
        cs=cast.split(':')
        colname = extract_df[number].columns[int(cs[0])]
        extract_df[number]=extract_df[number].rename({colname:cs[1]},axis='columns')
    #get_coloumns_info(number)
    return

#### insert an index column to a dataframe


In [11]:
def insert_column(number):
    extract_df[number].insert(loc=0,column='t_index',value=0)
    extract_df[number]['t_index']=range(1,len(extract_df[number])+1)
    return

#### insert an index column to a dataframe


In [12]:
import io
def road_map(edition):
    with open(path+'road_map'+edition+'.txt', 'w') as f:
        for index,table in extract_table_names.iterrows():
            print('Table Name: ',index,'   ',table[0],file=f)
            print('--------------------------------------------------','\n\nTable Info:',file=f)
            print('-------------------------',file=f)
            buffer = io.StringIO()
            extract_df[index].info(buf=buffer)
            s = buffer.getvalue()
            f.write(s)
            count_nan = len(extract_df[index]) - extract_df[index].count()
            colname=[]
            for index1,x in enumerate(count_nan):
                if x!=0:
                    colname.append(extract_df[index].columns[index1])
            if len(colname)>=1:
                print('\nColumn/s Describition\n','-------------------------',file=f)
                x=extract_df[index][colname].describe()
                print(x,file=f)        
            print('\n\n',file=f)
    
    return

#### Creat draft text file to use for EDR generation


In [13]:
def create_EDR():
    sql_table_name=pd.read_csv(path+'sql_dest_tables.txt',header=None)  
    for index,line in sql_table_name.iterrows():      
        if line[0].find('#')>=0:            
            print('\n',line[0].replace("#", ''))
            print('-------------')
        else:
            col=line[0].split('.')
            colname=extract_df[int(col[0])].columns[int(col[1])]
            key=''
            if (extract_df[int(col[0])].dtypes[colname]=='object'):
                temp='VARCHAR(100)'
            elif (extract_df[int(col[0])].dtypes[colname]=='int64'):
                temp='INT'
            elif (extract_df[int(col[0])].dtypes[colname]=='float64'):
                temp='FLOAT'
            final_text=extract_df[int(col[0])].columns[int(col[1])]
            final_text=final_text.replace(' ','_')
            if len(col)>=3:
                if col[2]=='p':
                    key='PK'            
            print(final_text,' ',key,' ',temp)
    return

#### Copy unique coulmn values from one df to another df


In [14]:
def move_data(table1,column1,table2,column2):
    extract_df[table2][extract_df[table2].columns[column2]]=extract_df[table1][extract_df[table1].columns[column1]].unique()
    return  

#### Function to create extra two df


In [15]:
def create_df(table_code):
    if table_code==1:
        extract_table_names.loc[4]='Regional'
        extract_df.append(pd.DataFrame(columns=['Regional_name']))        
        move_data(3,1,4,0)
    if table_code==2:
        extract_table_names.loc[5]='yearly'
        extract_df.append(pd.DataFrame(columns=['yearly_id','past_year']))
        for i in range(1,100):
            extract_df[5] = extract_df[5].append({'yearly_id': i,'past_year':1950+i}, ignore_index=True)                                  
    return

#### Join two df depending on column name


In [16]:
def join(table1,column1,table2,column2):
    extract_df[table1]=pd.merge(extract_df[table2], extract_df[table1], how="outer", on=extract_df[table2].columns[column2])    
    return

#### User interface that we have used for testing purposes 


ix=0
while ix!=10:
    print("1 to dispaly a sorce table")
    print("2 to dispaly the columns information")
    print("3 to dispaly the table heatmap")
    print("4 replace nill values by 0")
    print("5 to drop columns")
    print("6 cahnge a coulmn type")
    print("7 rename a coulmn ")
    print("8 insert index column")
    print("9 create road map")
    
    print("please select a number")

    ix=int(input())
    clear_output(wait=True)
    if ix==1: 
        number=int(input("please enter the table number"))
        print_extract_df(number)
    if ix==2: 
        number=int(input("please enter the table number"))
        get_coloumns_info(number)
    if ix==3: 
        number=int(input("please enter the table number"))
        check_extarct_data(number)
    if ix==4: 
        number=int(input("please enter the table number"))
        replace_none(number)   
    if ix==5: 
        number=int(input("please enter the table number"))
        cols=int(input("please enter a list of column/s"))
        print(cols)
        drop_columns(number,cols)   
    if ix==6: 
        number=int(input("please enter the table number"))
        print("example: 0:int64,5:object,3:datetime64")
        data=input()
        replace_column_type(number,data)   
    if ix==7: 
        number=int(input("please enter the table number"))
        print("example: 0:John,1:Hatim,3:Hayley,4:Tuflikha")
        data=input()
        rename_column(number,data)
    if ix==8: 
        number=int(input("please enter the table number"))
        insert_column(number)
    if ix==9: 
        number=input("please enter road map edition number")
        road_map(number)
    



#### automated task for the full ETL process, a sequence of functions calling

In [17]:
road_map('1')
#create regionla table
create_df(1)
insert_column(4)
rename_column(4,'0:Regional_id')

#creat yearly table
create_df(2)

#create index and rename columns name for country code
insert_column(0)
rename_column(0,'0:country_id,1:english_name,2:alpha_2_code,3:alpha_3_code,4:numeric_code,5:iso_3166_2') 
rename_column(2,'0:english_name,1:yearly_id,2:life_ladder,3:log_gdp_per_capita,4:social_support,\
                5:healthy_life_expectancy_at_birth,6:freedom_to_make_life_choices,7:generosity,\
              8:perceptions_of_corruption') 
rename_column(1,'0:english_name,1:population,3:net_change,6:migrants,7:fert_rate,8:med_age,9:urban_pop')
rename_column(3,'0:english_name,1:regional_name,2:life_ladder,6:log_gdp_per_capita')


In [18]:
replace_none(0)
replace_none(1)
replace_none(2)
sub=extract_df[1][['english_name','Land Area (Km²)']]
extract_df[0]=pd.merge(extract_df[0], sub, how="left", on=['english_name'])
replace_none(0)
rename_column(0,'6:land_area') 
sub=extract_df[0][['country_id','english_name']]
extract_df[1]=pd.merge(extract_df[1], sub, how="left", on=['english_name'])

the table code is (0) the table name is: wikipedia-iso-country-codes.csv
the table code is (1) the table name is: population_by_country_2020.csv
the table code is (2) the table name is: world-happiness-report.csv
the table code is (0) the table name is: wikipedia-iso-country-codes.csv


In [19]:
extract_df[1] = extract_df[1][extract_df[1]['country_id'] >= 1]
drop_columns(1,[10,5,4,1,0])
extract_df[1]['yearly_id']=2020-1950
extract_df[2]['yearly_id']=extract_df[2]['yearly_id']-1950
extract_df[2]=pd.merge(extract_df[2], sub, how="left", on=['english_name'])
drop_columns(2,[10,9,0])
extract_df[3]['yearly_id']=2021-1950
extract_df[3]=pd.merge(extract_df[3], sub, how="left", on=['english_name'])
extract_df[3] = extract_df[3][extract_df[3]['country_id'] >= 1]
drop_columns(3,[19,18,17,16,15,14,13,12,5,4,3,1,0])
rename_column(3,'2:social_support,3:healthy_life_expectancy_at_birth,4:freedom_to_make_life_choices,5:generosity,\
              6:perceptions_of_corruption') 
extract_df[2]=pd.concat([extract_df[2], extract_df[3]])
extract_df[2]
extract_df[2] = extract_df[2][extract_df[2]['country_id'] >= 1]
type(extract_table_names)
print(extract_table_names)
extract_table_names.drop([3,4], inplace=True)
print(extract_table_names)
extract_df.pop(3)
extract_df.pop(3)
extract_table_names.reset_index(inplace=True)
print(extract_table_names)
drop_columns(1,[0])
road_map('2')


the table code is (1) the table name is: population_by_country_2020.csv
the table code is (2) the table name is: world-happiness-report.csv
the table code is (3) the table name is: world-happiness-report-2021.csv
                                 0
0  wikipedia-iso-country-codes.csv
1   population_by_country_2020.csv
2       world-happiness-report.csv
3  world-happiness-report-2021.csv
4                         Regional
5                           yearly
                                 0
0  wikipedia-iso-country-codes.csv
1   population_by_country_2020.csv
2       world-happiness-report.csv
5                           yearly
   index                                0
0      0  wikipedia-iso-country-codes.csv
1      1   population_by_country_2020.csv
2      2       world-happiness-report.csv
3      5                           yearly
the table code is (1) the table name is: population_by_country_2020.csv


### Connect to local database

In [20]:

rds_connection_string = "postgres:postgres@localhost:5432/ETL-Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [21]:
engine.table_names()

<ipython-input-21-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['country_code', 'world_happiness', 'yearly', 'population_by_country']

### Use pandas to load csv converted DataFrame into database

In [22]:
extract_df[3].to_sql(name='yearly', con=engine, if_exists='append', index=False)
extract_df[0].to_sql(name='country_code', con=engine, if_exists='append', index=False)
extract_df[1].to_sql(name='population_by_country', con=engine, if_exists='replace', index=False)
extract_df[2].to_sql(name='world_happiness', con=engine, if_exists='replace', index=False)

### Confirm data has been added by querying to all the tables


In [23]:
pd.read_sql_query('select * from country_code', con=engine).head()

,country_id,english_name,alpha_2_code,alpha_3_code,numeric_code,iso_3166_2,land_area
0,1,Afghanistan,AF,AFG,4,ISO 3166-2:AF,652860
1,2,Åland Islands,AX,ALA,248,ISO 3166-2:AX,0
2,3,Albania,AL,ALB,8,ISO 3166-2:AL,27400
3,4,Algeria,DZ,DZA,12,ISO 3166-2:DZ,2381740
4,5,American Samoa,AS,ASM,16,ISO 3166-2:AS,200


In [24]:
pd.read_sql_query('select * from yearly', con=engine).head()

,yearly_id,past_year
0,1,1951
1,2,1952
2,3,1953
3,4,1954
4,5,1955


In [25]:
pd.read_sql_query('select * from population_by_country', con=engine).head()

,net_change,migrants,fert_rate,med_age,urban_pop,country_id,yearly_id
0,5540090,-348399.0,1.7,38,61 %,45.0,70
1,13586631,-532687.0,2.2,28,35 %,101.0,70
2,1937734,954806.0,1.8,38,83 %,233.0,70
3,2898047,-98955.0,2.3,30,56 %,102.0,70
4,4327022,-233379.0,3.6,23,35 %,167.0,70


In [26]:
pd.read_sql_query('select * from world_happiness', con=engine).head()

,yearly_id,life_ladder,log_gdp_per_capita,social_support,healthy_life_expectancy_at_birth,freedom_to_make_life_choices,generosity,perceptions_of_corruption,country_id
0,58,3.724,7.370,0.451,50.80,0.718,0.168,0.882,1.0
1,59,4.402,7.540,0.552,51.20,0.679,0.190,0.850,1.0
2,60,4.758,7.647,0.539,51.60,0.600,0.121,0.707,1.0
3,61,3.832,7.620,0.521,51.92,0.496,0.162,0.731,1.0
4,62,3.783,7.705,0.521,52.24,0.531,0.236,0.776,1.0
